In [ ]:
library(httr)
library(data.table)
library(dplyr)
library(jsonlite)
getwd()
CTS_V2_API_KEY <- Sys.getenv("CTS_V2_API_KEY")


#### Setup


In [ ]:
PAGE_SIZE <- 50
INCLUDE_FIELDS <- c(
  "nct_id",
  "diseases",
  "current_trial_status",
  "primary_purpose",
  "sites.recruitment_status"
)

poc_disease_search <- function(
    ncit_code,
    from = 0,
    size = PAGE_SIZE) {
  response <- httr::POST(
    "https://clinicaltrialsapi.cancer.gov/api/v2/trials",
    # Copied from get_api_studies_for_disease.R
    body = list(
      current_trial_status = "Active",
      primary_purpose = c("TREATMENT", "SCREENING"),
      # maintype = c("C4872"),
      # stage = ncit_code,
      diseases.nci_thesaurus_concept_id = ncit_code,
      sites.recruitment_status = "ACTIVE",
      include = INCLUDE_FIELDS,
      from = from,
      size = size
    ),
    encode = "json",
    httr::add_headers(`x-api-key` = CTS_V2_API_KEY, `Content-Type` = "application/json"),
    httr::timeout(5)
  )
  data <- httr::content(response)
  assertthat::assert_that(response$status_code == 200, msg = paste("Response status is", response$status_code))
  return(data)
}

paginate_cts_api <- function(paged_data, total_expected, FUN, ...) {
  print(paste("    Expecting:", total_expected))
  while (length(paged_data) < total_expected) {
    data <- FUN(from = length(paged_data), ...)
    paged_data <- append(paged_data, data$data)
    print(paste("          Got:", length(data$data)))
    print(paste("        Total:", length(paged_data)))
  }
  return(paged_data)
}


#### Fetch trials from CTS API


In [ ]:
search_term <- "Stage IA Breast Cancer"
breast_carcinoma <- c("C4872")
# ncit_code <- c("C4872") # Breast Carcinoma
# ncit_code <- c("C153238") # Metastatic Breast Carcinoma
# ncit_code <- c("C3641") # Stage 0 Breast Cancer AJCC v6 and v7
ncit_code <- c("C85835", "C139557", "C139536") # Stage IA BC (w/ AJCC versions)
trials_p1 <- poc_disease_search(ncit_code)


In [ ]:
all_trials <- paginate_cts_api(trials_p1$data, trials_p1$total, poc_disease_search, ncit_code = ncit_code)


#### Prepare the diseases


In [ ]:
count <- 0
disease_count <- 0
list_of_trial_disease_list <- Map(function(t) {
  count <<- count + 1
  diseases <- Map(function(d) {
    disease_count <<- disease_count + 1
    return(
      list(
        nct_id = t$nct_id,
        disease = d$name,
        inclusion_indicator = d$inclusion_indicator,
        code = d$nci_thesaurus_concept_id,
        type = toString(d$type),
        parents = toString(d$parents)
      )
    )
  }, t$diseases)
  return(diseases)
}, all_trials)
list_of_trial_disease_dt <- lapply(list_of_trial_disease_list, rbindlist)
count
disease_count


#### Check the diseases


In [ ]:
#' Make sure that the search term appears in each trial
noop <- lapply(list_of_trial_disease_dt, function(trial_disease_dt) {
  assertthat::assert_that(search_term %in% trial_disease_dt$disease)
})

#' Apply the following if checking leaf-node searches (e.g. Stage search with no children terms)
noop <- lapply(list_of_trial_disease_dt, function(trial_disease_dt) {
  search_term_idxs <- which(trial_disease_dt$disease == search_term)
  terms <- trial_disease_dt[search_term_idxs]
  assertthat::assert_that(length(terms) >= 1,
    msg = "Expecting at least one instance of search_term"
  )
  assertthat::assert_that(unique(terms$inclusion_indicator) == "TRIAL",
    msg = "Expecting that leaf-node terms should all be found at TRIAL level"
  )
})

diseases_df <- rbindlist(list_of_trial_disease_dt)


#### View relationship of `type`

In [ ]:
diseases_df[diseases_df$code %in% c(breast_carcinoma, ncit_code), c("code", "disease", "type")] %>%
  arrange(type, disease) %>%
  unique()


#### First pass at an approach for searching up the tree

In [194]:
visited <- list()
paths <- list()

construct_paths_to_root <- function(trial_disease_dt, starting_terms) {
  terms <- trial_disease_dt[trial_disease_dt$code %in% starting_terms, ]

  apply(terms, 1, function(term) {
    term <- as.list(term)
    if (!term["code"] %in% visited) {
      visited <<- c(visited, term["code"])
      paths <<- append(paths, list(code = term["code"], disease = term["disease"]))
    }
  })

  # For each term's parents
  lapply(terms$parents, function(p_chr) {
    assertthat::assert_that(length(p_chr) == 1, msg = "Parents character vector should have length of 1")
    p_list <- strsplit(p_chr, split = ",\\s*")
    # Apply the same path construction to each parent
    lapply(p_list[[1]], function(p_chr) {
      if (!p_chr %in% visited) {
        construct_paths_to_root(trial_disease_dt, starting_terms = p_chr)
      }
    })
  })
}

noop <- lapply(list_of_trial_disease_dt, function(dt) construct_paths_to_root(dt, ncit_code))
paths


$code
$code$code
[1] "C139536"


$disease
$disease$disease
[1] "Stage IA Breast Cancer"


$code
$code$code
[1] "C139535"


$disease
$disease$disease
[1] "Stage I Breast Cancer"


$code
$code$code
[1] "C139533"


$disease
$disease$disease
[1] "Breast Cancer by AJCC v8 Anatomic Stage"


$code
$code$code
[1] "C139532"


$disease
$disease$disease
[1] "Breast Cancer by AJCC v8 Stage"


$code
$code$code
[1] "C4872"


$disease
$disease$disease
[1] "Breast Cancer"


$code
$code$code
[1] "C9335"


$disease
$disease$disease
[1] "Malignant Breast Neoplasm"


$code
$code$code
[1] "C2910"


$disease
$disease$disease
[1] "Breast Neoplasm"


$code
$code$code
[1] "C26709"


$disease
$disease$disease
[1] "Breast Disorder"


$code
$code$code
[1] "C27551"


$disease
$disease$disease
[1] "Disorder by Site"


$code
$code$code
[1] "C2991"


$disease
$disease$disease
[1] "Other Disease"


$code
$code$code
[1] "C3263"


$disease
$disease$disease
[1] "Neoplasm by Site"


$code
$code$code
[1] "C3262"


$disease
$disease$disease
[1] "Other Neoplasm"


$code
$code$code
[1] "C9305"


$disease
$disease$disease
[1] "Malignant Neoplasm"


$code
$code$code
[1] "C7062"


$disease
$disease$disease
[1] "Neoplasm by Special Category"


$code
$code$code
[1] "C2916"


$disease
$disease$disease
[1] "Other Carcinoma"


$code
$code$code
[1] "C3709"


$disease
$disease$disease
[1] "Epithelial Neoplasm"


$code
$code$code
[1] "C4741"


$disease
$disease$disease
[1] "Neoplasm by Morphology"


$code
$code$code
[1] "C85835"


$disease
$disease$disease
[1] "Stage IA Breast Cancer"


$code
$code$code
[1] "C88375"


$disease
$disease$disease
[1] "Stage I Breast Cancer"


$code
$code$code
[1] "C91230"


$disease
$disease$disease
[1] "Breast Cancer by AJCC v7 Stage"


$code
$code$code
[1] "C139557"


$disease
$disease$disease
[1] "Stage IA Breast Cancer"


$code
$code$code
[1] "C139556"


$disease
$disease$disease
[1] "Stage I Breast Cancer"


$code
$code$code
[1] "C139554"


$disease
$disease$disease
[1] "Breast Cancer by AJCC v8 Prognostic Stage"